### Putting it all together

In [ ]:
import getpass
import os
import bs4
import tiktoken
from langchain_community.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


api_key = "YOUR OPENAI API KEY"
os.environ["OPENAI_API_KEY"] = api_key


loader = PyPDFLoader("YOUR_PDF_LOCATION.pdf")
docs = loader.load()

#print(f"Total characters: {len(docs[0].page_content)}")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200, add_start_index = True
)
all_splits = text_splitter.split_documents(docs)

#print(f"Split blog post into {len(all_splits)} sub-documents.")


embeddings = OpenAIEmbeddings(openai_api_key = api_key)
vector_store = InMemoryVectorStore(embeddings)
document_ids =  vector_store.add_documents(documents = all_splits)


PROMPT_TEMPLATE = """
You are a highly skilled researcher with expertise in machine learning, advanced mathematics, and statistical modeling. 
Your responses should demonstrate a deep understanding of these domains, providing precise, 
well-explained answers that include relevant mathematical formulations, 
citations to foundational concepts, and real-world examples where appropriate. 
When uncertain, admit the lack of specific knowledge rather than speculating. 
Aim for a formal tone, as your audience consists of academics and professionals in these fields. 
Structure responses clearly, starting with an overview before delving into technical details.

Conversation History: {history}
Context: {context}
Question: {question}
Answer the question based on the above context and history:"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)


llm = ChatOpenAI(model = "gpt-4o")
# Set token limit for the model
TOKEN_LIMIT = 4000

conversation_history = []

def count_tokens(text, model="gpt-4o"):
    """Estimate the number of tokens in a given text."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def truncate_history(conversation_history, max_tokens):
    """Truncate conversation history to fit within token limits."""
    while count_tokens("\n".join([f"User: {msg['user']}\nAssistant: {msg['assistant']}" for msg in conversation_history])) > max_tokens:
        conversation_history.pop(0)  # Remove the oldest interaction
    return conversation_history

def chatbot(question: str):
    global conversation_history

    # Retrive relevant context
    retrieved_docs = vector_store.similarity_search(question)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # Format the conversation history
    formatted_history = "\n".join(
        [f"User: {msg['user']}\nAssistant: {msg['assistant']}" for msg in conversation_history]
        )

    # Create the full prompt
    prompt = prompt_template.format(
        history=formatted_history, context=docs_content, question=question
    )
    
    # Check token count and truncate if necessary
    total_tokens = count_tokens(prompt)

    if total_tokens > TOKEN_LIMIT:
        conversation_history = truncate_history(conversation_history, TOKEN_LIMIT - count_tokens(docs_content) - 500)
        formatted_history = "\n".join(
            [f"User: {msg['user']}\nAssistant: {msg['assistant']}" for msg in conversation_history]
        )
        prompt = prompt_template.format(
            history=formatted_history, context=docs_content, question=question
        )

    # Generate a response
    try:
        response = llm.invoke(prompt)
        answer = response.content

        # Update conversation history
        conversation_history.append({"user": question, "assistant": answer})

        return answer
    
    except Exception as e:
        print (f'Error: {e}')
        return "Sorry, the system encountered error while processing your request."

# Example Chatbot Interaction
print("Chatbot Initialized! Ask a question or type 'exit' to quit.")
while True:
    user_question = input("\nYou: ")
    if user_question.lower() == "exit":
        print("Exiting chatbot. Goodbye!")
        break
    response = chatbot(user_question)

    print(f"\nYou: {user_question}")
    print(f"Assistant: {response}")